<a href="https://www.kaggle.com/code/luisgasparcordeiro/football-champions-by-season?scriptVersionId=115330716" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import warnings
warnings.filterwarnings('ignore')

/kaggle/input/player-scores/players.csv
/kaggle/input/player-scores/competitions.csv
/kaggle/input/player-scores/games.csv
/kaggle/input/player-scores/game_events.csv
/kaggle/input/player-scores/club_games.csv
/kaggle/input/player-scores/player_valuations.csv
/kaggle/input/player-scores/appearances.csv
/kaggle/input/player-scores/clubs.csv


# 0 - Data Preparation


In [2]:
# Read CSVs into dataframes
competitions = pd.read_csv("/kaggle/input/player-scores/competitions.csv")
games = pd.read_csv("/kaggle/input/player-scores/games.csv")
club_games = pd.read_csv("/kaggle/input/player-scores/club_games.csv")
appearances = pd.read_csv("/kaggle/input/player-scores/appearances.csv")
clubs = pd.read_csv("/kaggle/input/player-scores/clubs.csv")
finals = games[(games["round"] == "Final")]

In [3]:
matchdays = games[(games["round"].str.contains("Matchday"))]
matchdays["round"] = matchdays["round"].str.split(".").str.get(0)
matchdays['round'] = matchdays['round'].astype('int')
last_matchday = matchdays.groupby(["competition_id","season"],as_index=False).max("round")
last_matchday = last_matchday[["competition_id","season","round"]]
finals = finals.groupby(["competition_id","season", "round"],as_index=False).sum("home_club_goals")
finals = finals[["competition_id","season", "round"]]
competition = pd.concat([finals,last_matchday])
competition = competition.drop_duplicates().reset_index().drop("index", axis=1)

# 1 - Champions
Let's create a function that returns a list of champions of a given competition

In [4]:
def league_champions(league):
    
    from pandasql import sqldf
    pysqldf = lambda q: sqldf(q, globals())

    q = f"""

          SELECT 
            g.season,
            case when g.home_club_position = 1 then g.club_home_pretty_name 
                 when g.away_club_position = 1 then g.club_away_pretty_name 
                 end as Champion
           FROM games g
           LEFT JOIN competition c
               on c.season = g.season
               and c.competition_id = g.competition_id
           where 
            cast(SUBSTRING(g.round, 1,2) as int) = c.round
           and g.competition_id = '{league}'
           and (g.home_club_position = 1 or g.away_club_position = 1)

           ;"""

    champions = pysqldf(q)
    return champions

def cup_champions(league):
    
    from pandasql import sqldf
    pysqldf = lambda q: sqldf(q, globals())

    q = f"""

          SELECT 
            g.season,
            case when g.home_club_goals > g.away_club_goals then g.club_home_pretty_name
                 when g.home_club_goals < g.away_club_goals then g.club_away_pretty_name
                 when g.home_club_goals = g.away_club_goals then "draw"
                 else "UNK"
                 end as Champion
           FROM games g
           LEFT JOIN competition c
               on c.season = g.season
               and c.competition_id = g.competition_id
           where
           g.competition_id = '{league}'
           and g.round = c.round
            
           ;"""

    champions = pysqldf(q)
    return champions

def champions(league):
    
    cups = competition[competition["round"] == "Final"]["competition_id"].unique()
    leagues = competition[competition["round"] != "Final"]["competition_id"].unique()
    
    if league in leagues: 
        return league_champions(league)

    if league in cups: 
        return cup_champions(league)
    
    else:
        return "Competition ID not found"

In [5]:
print("Valid Competititons:")
print(games["competition_id"].unique())

Valid Competititons:
['DFL' 'NLSC' 'DFB' 'BE1' 'L1' 'NL1' 'ES1' 'SUC' 'BESC' 'FR1' 'CDR' 'CGB'
 'GB1' 'POCP' 'NLP' 'FRCH' 'PO1' 'RU1' 'SC1' 'UKRS' 'SFA' 'UKRP' 'GBCS'
 'RUP' 'RUSS' 'UKR1' 'IT1' 'GR1' 'DKP' 'USC' 'SCI' 'GRP' 'CIT' 'EL' 'TR1'
 'CL' 'DK1' 'KLUB' 'CLQ' 'ELQ' 'POSU' 'ECLQ']


In [6]:
# Italian Champions (Seria A)
champions("IT1")

,season,Champion
0,2012,Juventus Turin
1,2013,Juventus Turin
2,2014,Juventus Turin
3,2015,Juventus Turin
4,2016,Juventus Turin
5,2017,Juventus Turin
6,2018,Juventus Turin
7,2020,Juventus Turin
8,2019,Juventus Turin
9,2020,Inter Mailand


# 2 - League Position
Lets build a function to check in which place did a given club finished their domestic league, in a given season

In [7]:
def league_position(club, season):
    
    from pandasql import sqldf
    pysqldf = lambda q: sqldf(q, globals())
    
    
    domestic_league = clubs[clubs['pretty_name'] == club]["domestic_competition_id"].values[0]

    q = f"""

          SELECT 
            '{club}' as Club_name,
            g.season,
            g.competition_id,
            case when g.club_home_pretty_name = '{club}' then g.home_club_position 
                 when g.club_away_pretty_name = '{club}' then g.away_club_position 
                 else "Error"
                 end as Position
           FROM games g
           LEFT JOIN competition c
               on c.season = g.season
               and c.competition_id = g.competition_id
           where 
           cast(SUBSTRING(g.round, 1,2) as int) = c.round
           and g.season = {season}
           and g.competition_id = '{domestic_league}'
           and (g.club_home_pretty_name = '{club}' or g.club_away_pretty_name = '{club}')

           ;"""

    champions = pysqldf(q)
    return champions


In [8]:
# Manchester City's final stand in 2021/2022 season
league_position('Manchester City', 2021)

,Club_name,season,competition_id,Position
0,Manchester City,2021,GB1,1


Good! The above method works!

But it is a bit slow - and that is because it is using a SQL query.

We can make one that runs quicker if we use native pandas commands only

In [9]:
def league_position_py(club, season):
    
    domestic_league = clubs[clubs['pretty_name'] == club]["domestic_competition_id"].values[0]

    # Merge the two dataframes on the season and competition_id columns
    df = games.merge(competition, on=['season', 'competition_id'], how='left')

    df = df[df["competition_type"] == "domestic_league"]
    df["round_x"] = df["round_x"].str.split(".").str[0].astype('int')
    df["round_y"] = df["round_y"].astype('int')
    df = df[df["round_y"] == df["round_x"]]
    df = df[(df['season'] == season) & (df['competition_id'] == domestic_league) ]
    df = df[(df['club_away_pretty_name'] == club) | (df['club_home_pretty_name'] == club)]
    # Select the relevant columns
    df = df[['season', 'competition_id', 'club_home_pretty_name', 'home_club_position', 
             'club_away_pretty_name', 'away_club_position']]

    df['Position'] = df.apply(lambda row: row['home_club_position'] if row['club_home_pretty_name'] == club else row['away_club_position'] if row['club_away_pretty_name'] == club else 'Error', axis=1)
    df['Club_name'] = df.apply(lambda row: row['club_home_pretty_name'] if row['club_home_pretty_name'] == club else row['club_away_pretty_name'] if row['club_away_pretty_name'] == club else 'Error', axis=1)

    return df[["Club_name","season","competition_id","Position"]]

In [10]:
# Manchester City's final stand in 2021/2022 season
league_position_py('Manchester City', 2021)

,Club_name,season,competition_id,Position
50419,Manchester City,2021,GB1,1


### Looks Consistent! - and it is quicker

# 3 - Champions League

"Let's create a function that retrieves the highest stage that a club has reached in the Champions League for a specific season."

In [11]:
def champions_league_stage(club, season):

    CL = games[(games['competition_id'] == 'CL') & (games['season'] == season )]
    CL = CL[(CL['club_away_pretty_name'] == club) | (CL['club_home_pretty_name'] == club)]
    
    if (CL.shape[0] == 0):
        
        Out = [(club, season, 'CL', 'Absent OR Invalid args')]
        a = pd.DataFrame(data = Out, columns = ['Club_name','season','competition_id','Position'])
        return a
        
    else:    
        
        last_game = CL[CL['date'] == CL.date.max()]

        last_game['Stage'] = np.select([(( (last_game['round'] == ('Final'))
                                      & (last_game['club_home_pretty_name']==club)
                                      & (last_game['home_club_goals'] > last_game['away_club_goals'] )) |
                                      ( (last_game['round'] == ('Final'))
                                      & (last_game['club_away_pretty_name']==club)
                                      & (last_game['home_club_goals'] < last_game['away_club_goals']) )),
                                      last_game['round'] == ('Final'),
                                      last_game['round'].str.contains('Semi-Finals'),
                                      last_game['round'].str.contains('Quarter-Finals'),
                                      last_game['round'].str.contains('last 16'),
                                      last_game['round'].str.contains('Group'),]
                                     ,choicelist=['Winner','Final', 'Semi-Finals', 'Quarter-Finals', 'Last 16', 'Group Stage'])

        last_game['Club_name'] = last_game.apply(lambda row: row['club_home_pretty_name'] if row['club_home_pretty_name'] == club else row['club_away_pretty_name'] if row['club_away_pretty_name'] == club else 'Error', axis=1)

        return last_game[['Club_name','season','competition_id','Stage']]

In [12]:
champions_league_stage('Real Madrid', 2021)

,Club_name,season,competition_id,Stage
55424,Real Madrid,2021,CL,Winner


## Looks Correct!